In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from requests import get
import time
import pickle
import os
import json
from typing import Dict, List
import requests
import pprint
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud



import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

import prep
from env import github_token

## Planning

## Acquire 

Acuired most starred repos using BeautifulSoup to scrape repo links from the first 22 pages. Then used the Github API to pull the README.md files for each repo.

In [2]:
#grabs list of repo names from most starred repo pages.
def get_urls(start_page,stop_page):
    pages = list(range(start_page,stop_page+1))
    page_urls = []
    for page in pages:
        p = f'https://github.com/search?o=desc&p={page}&q=stars%3A%3E1&s=stars&type=Repositories'
        page_urls.append(p)

    repo_links_all = []
    time.sleep(3)
    for page in page_urls:
        url = page
        headers  = {'User-Agent' : 'Codeup Data Science Student'} #user-agent to decrease chance of being rejected by github
        response = get(url,headers)
        soup = BeautifulSoup(response.text) #created a BS object
        repo_list = soup.find('ul', class_='repo-list') #find section of text of interest
        links = repo_list.find_all(href = True) #isolate the 'href's 
        repo_links= [link['href'] for link in links if not re.match(r'^/topics',link['href']) and not re.match(r'^https',link['href'])\
                 and link['href'].count('/') < 3] #further pare down list by excluding anything that begins with /topics, https or more than 2 forward slashes
        repo_links_all.extend(repo_links)
    repo_links_all = [url[1:] for url in repo_links_all]
    time.sleep(2)
    return repo_links_all

We had problems scraping the website with the rest of the class also pinging the Github servers so we decided to save our urls and READMEs as files to be accessed locally

In [3]:
with open ('urls', 'rb') as fp:
    urls = pickle.load(fp)

In [4]:
original = pd.read_json('data.json')
original.head(2)

,language,readme_contents,repo
0,JavaScript,![freeCodeCamp.org Social Banner](https://s3.a...,freeCodeCamp/freeCodeCamp
1,Rust,[996.ICU](https://996.icu/#/en_US)\n=======\n*...,996icu/996.ICU


### Initial Exploration

We pulled 220 initial README files and grouped by the language. 32 had no language type and were dropped. After further discussion we decided to only work on classifying the top 5 most common languages of our data.

In [5]:
pprint.pprint(original.groupby('language')['repo'].count().sort_values(ascending=False).head(8))

language
JavaScript    72
Python        25
Java          17
C++           12
Go            12
TypeScript     9
CSS            5
C              4
Name: repo, dtype: int64


### Data Prep

Prepared data with cleaned data (lower case, only ascii, and no special characters), stemmed, and lemmatized

In [6]:
df = prep.prep_urls(original)

In [7]:
df.head(2)

,language,readme_contents,repo,original,clean,stemmed,lemmatized
0,JavaScript,![freeCodeCamp.org Social Banner](https://s3.a...,freeCodeCamp/freeCodeCamp,![freeCodeCamp.org Social Banner](https://s3.a...,freecodecamporg social bannerhttpss3amazonawsc...,freecodecamporg social bannerhttpss3amazonawsc...,freecodecamporg social bannerhttpss3amazonawsc...
1,Rust,[996.ICU](https://996.icu/#/en_US)\n=======\n*...,996icu/996.ICU,[996.ICU](https://996.icu/#/en_US)\n=======\n*...,996icuhttps996icuenus please note exists offic...,996icuhttps996icuenu pleas note exist offici a...,996icuhttps996icuenus please note exists offic...


- set the index to the repo name. 
- Only look at the top 5 most common languages
- Dropped readme_contents column

In [8]:
df.set_index('repo',inplace=True)
languages = ['JavaScript','Python','Java','C++','Go']
df = df[df.language.isin(languages)]
df.drop(columns='readme_contents')
df.head(2)

,language,readme_contents,original,clean,stemmed,lemmatized
repo,,,,,,
freeCodeCamp/freeCodeCamp,JavaScript,![freeCodeCamp.org Social Banner](https://s3.a...,![freeCodeCamp.org Social Banner](https://s3.a...,freecodecamporg social bannerhttpss3amazonawsc...,freecodecamporg social bannerhttpss3amazonawsc...,freecodecamporg social bannerhttpss3amazonawsc...
vuejs/vue,JavaScript,"<p align=""center""><a href=""https://vuejs.org"" ...","<p align=""center""><a href=""https://vuejs.org"" ...",p aligncentera hrefhttpsvuejsorg targetblank r...,p aligncentera hrefhttpsvuejsorg targetblank r...,p aligncentera hrefhttpsvuejsorg targetblank r...


32 repos had no listed language and were dropped from our dataset leaving us with a dataset of 138 repos.

In [9]:
df_nulls = df[df.language.isnull()]
df = df.dropna(axis=0)

In [10]:
df.shape

(138, 6)

### Exploration

Sean add cool stuff here plz

### Beating the Baseline

#### Logistic Regression

Baseline model attempting to beat highest % language occurance (Javascript 52%). Uses Logistic regression to classify each programming language based on the content of the cleaned corpus words.

In [11]:
df = df[['clean','language']]
df.rename(columns={'clean':'body'},inplace=True)

In [12]:
tfidf = TfidfVectorizer()
X = tfidfs = tfidf.fit_transform(df.body)
y = df.language

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=.3)

In [14]:
train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

lm = LogisticRegression().fit(X_train,y_train)

In [15]:
train['predicted_LR'] = lm.predict(X_train)
test['predicted_LR'] = lm.predict(X_test)

In [16]:
print("The train set predicted: ",round(sum(train.actual == train.predicted_LR)/len(train)*100),"%")
print("This was 7% better than the baseline of picking the most common language")

The train set predicted:  55 %
This was 7% better than the baseline of picking the most common language


In [17]:
print("The test set predicted: ",round(sum(test.actual == test.predicted_LR)/len(test)*100),"%")
print("This was 0% better than the baseline of picking the most common language")

The test set predicted:  52 %
This was 0% better than the baseline of picking the most common language


#### Random Forest Classifer

After this Logistic regression showed only a slight improvement on the test data a RF classifier was chosen to try to improve performance. Using a max depth of 3 we saw improvements on the train and test data. 

In [18]:
clf = RandomForestClassifier(max_depth = 3,random_state=123)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=3, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [19]:
train['predicted_RFC'] = clf.predict(X_train)
test['predicted_RFC'] = clf.predict(X_test)

In [20]:
print("The train set predicted: ",round(sum(train.actual == train.predicted_RFC)/len(train)*100),"%")
print("This was 9% better than the baseline of picking the most common language")

The train set predicted:  56 %
This was 9% better than the baseline of picking the most common language


In [21]:
print("The test set predicted: ",round(sum(test.actual == test.predicted_RFC)/len(test)*100),"%")
print("This was 3% better than the baseline of picking the most common language")

The test set predicted:  52 %
This was 3% better than the baseline of picking the most common language


In [22]:
#prediction of likelihood a corpus belongs to each language
preds = clf.predict_proba(X_train)
preds = pd.DataFrame(preds,columns=clf.classes_)
preds = preds.round(2)
train = train.reset_index()

In [23]:
train = pd.concat([train,preds],axis=1)

In [24]:
train.set_index('repo',inplace=True)
train.head(15)

,actual,predicted_LR,predicted_RFC,C++,Go,Java,JavaScript,Python
repo,,,,,,,,
tensorflow/models,Python,JavaScript,JavaScript,0.04,0.03,0.11,0.48,0.35
sahat/hackathon-starter,JavaScript,JavaScript,JavaScript,0.03,0.09,0.05,0.72,0.11
pallets/flask,Python,JavaScript,JavaScript,0.06,0.04,0.13,0.58,0.20
minimaxir/big-list-of-naughty-strings,Python,JavaScript,JavaScript,0.04,0.03,0.15,0.49,0.29
angular/angular.js,JavaScript,JavaScript,JavaScript,0.06,0.04,0.13,0.58,0.20
lodash/lodash,JavaScript,JavaScript,JavaScript,0.05,0.03,0.12,0.64,0.17
gohugoio/hugo,Go,JavaScript,JavaScript,0.04,0.28,0.14,0.43,0.11
tesseract-ocr/tesseract,C++,JavaScript,JavaScript,0.12,0.14,0.16,0.45,0.13
expressjs/express,JavaScript,JavaScript,JavaScript,0.10,0.03,0.09,0.63,0.15
